<a href="https://colab.research.google.com/github/linglongchen/colab/blob/master/nb/Qwen3_(0.6B)-GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installation

In [1]:
!pip install vllm==0.8.5.post1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.4/326.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.4/98.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/2

In [2]:
!pip install unsloth bitsandbytes accelerate xformers peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer blake3 fastapi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.6/307.6 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.2/129.2 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [3]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-0.6B-unsloth-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,  # LoRA 方式微调
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
INFO 08-14 15:44:31 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-14 15:44:31 [__init__.py:239] Automatically detected platform cuda.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.5: Fast Qwen3 patching. Transformers: 4.55.0. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/576M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
# 微调前测试
EOS_TOKEN = tokenizer.eos_token # 必须添加 EOS_TOken

alpaca_prompt="""

下面是描述一个任务，以一个输入然后提供一个回复，/no_think。

### Instruction:

{}

### Input:

{}

### Response:

{}

"""

FastLanguageModel.for_inference(model)

inputs = tokenizer(

[

 alpaca_prompt.format(

 "", # instruction

"我花了500块做了个智商测试怎么测出来只有30??", # input

 "", # output

 )+EOS_TOKEN

 ], return_tensors = "pt").to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)

# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)



下面是描述一个任务，以一个输入然后提供一个回复，/no_think。

### Instruction:



### Input:

我花了500块做了个智商测试怎么测出来只有30??

### Response:



<|im_end|>
<|im_start|>
<|im_start|>
</think>

回复：
智商测试通常通过标准的测验工具来评估，比如斯坦福双测法、韦氏认知测验等。500块的花费可能只是初始的费用，实际结果会根据测试的具体内容和评分标准来判断。如果只是花了500块，但测试结果却显示只有30，可能是因为测试的难度、评分标准或测试本身存在一些问题。建议再试一次或咨询专业人士以获得更准确的结果。<|im_end|>


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 32,  # LoRA缩放系数
    lora_dropout = 0.0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.8.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
EOS_TOKEN = tokenizer.eos_token # 必须添加 EOS_TOken

def formatting_prompts_func(examples):

  instructions = examples["instruction"]

  inputs = examples["input"]

  outputs = examples["output"]

  texts = []

  for instruction, input, output in zip(instructions, inputs, outputs):

    # 必须添加EOS_TOKEN,否则无限生成

    text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN

    texts.append(text)

  return {"text":texts,}

pass

from datasets import load_dataset

dataset = load_dataset("logicChen/llama3-test", split="train")

dataset = dataset.map(formatting_prompts_func, batched=True,)

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

ruozhiba_qa.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1498 [00:00<?, ? examples/s]

Map:   0%|          | 0/1498 [00:00<?, ? examples/s]

In [5]:
from datasets import load_dataset
raw_ds = load_dataset(
    "json",
    data_files = {"train": "msgType.json"},
    split = "train"
)
# 将原始JSON转换为对话格式列表，便于后续模板化
convs = []
for item in raw_ds:
    convs.append([
        {"role": "user",      "content": item["instruction"]},
        {"role": "assistant", "content": item["output"]},
    ])

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
from datasets import Dataset
from unsloth.chat_templates import standardize_sharegpt

# 将 list 转成 Dataset
raw_conv_ds = Dataset.from_dict({"conversations": convs})

standardized = standardize_sharegpt(raw_conv_ds)

chat_inputs = tokenizer.apply_chat_template(
    standardized["conversations"],
    tokenize = False,
)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/862 [00:00<?, ? examples/s]

In [7]:
import pandas as pd
from datasets import Dataset

df = pd.DataFrame({"text": chat_inputs})
train_ds = Dataset.from_pandas(df).shuffle(seed = 666)

In [8]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps = 200,          # 训练步数，调大一点，毕竟小模型微调起来挺快的
        learning_rate = 5e-5,
        warmup_steps = 100,
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 666,
        report_to = "none",
    )
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/862 [00:00<?, ? examples/s]

## 开始训练

In [9]:
trainer_stats = trainer.train()
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 862 | Num Epochs = 2 | Total steps = 200
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 20,185,088 of 616,235,008 (3.28% trained)


Step,Training Loss
5,8.009900
10,7.835200
15,7.770000
20,7.233300
25,6.878400
30,6.131600
35,5.791200
40,5.176400
45,4.797800
50,4.601500


TrainOutput(global_step=200, training_loss=3.435685453414917, metrics={'train_runtime': 362.8744, 'train_samples_per_second': 4.409, 'train_steps_per_second': 0.551, 'total_flos': 159267853172736.0, 'train_loss': 3.435685453414917})


In [ ]:
#测试微调后的模型

FastLanguageModel.for_inference(model)

input_0 = tokenizer(

    [

        alpaca_prompt.format(

           "", # instruction

           "我花了500块做了个智商测试怎么测出来只有30? /no_think", # input

           "", # output

      )

    ], return_tensors = "pt").to("cuda")

from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)


_ = model.generate(**input_0, streamer=text_streamer,max_new_tokens=128)



下面是描述一个任务，以一个输入然后提供一个回复，/no_think。

### Instruction:



### Input:

我花了500块做了个智商测试怎么测出来只有30? /no_think

### Response:



这可能是因为你的智商测试是针对成年人的，而你只是做了一个测试，还没有完成你的智商测试。所以，你的智商只有30可能是因为你还没有完成这个过程。

<|im_end|>


In [10]:
trainer.save_model("ruozhi-Qwen3-0.6B")  # 保存lora模型和分词器

In [11]:
def ask_catgirl(question):
  messages = [
    {"role" : "user", "content" : question}
]
  text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = False, # 思考模式
)

  from transformers import TextStreamer
  _ = model.generate(
      **tokenizer(text, return_tensors = "pt").to("cuda"),
      max_new_tokens = 256, # 输出长度
      temperature = 0.7, top_p = 0.8, top_k = 20,
      streamer = TextStreamer(tokenizer, skip_prompt = True),
  )

In [12]:
ask_catgirl("TASK_GSDP_SORTINGCENTER_UNREACHABLE_RETURN_PROCESS")

不可达回传GSDP-大包回传<|im_end|>


In [14]:
from unsloth import FastLanguageModel

# Merge LoRA adapter with the base model
model.save_pretrained_merged("qwen3-0.6b-lora-merged", tokenizer, save_method = "merged_4bit_forced",)

Unsloth: Merging LoRA weights into 4bit model...


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Unsloth: Merging finished.
Unsloth: Found skipped modules: ['model.layers.0.self_attn.q_proj', 'model.layers.0.self_attn.k_proj', 'model.layers.0.self_attn.v_proj', 'model.layers.0.self_attn.o_proj', 'model.layers.2.mlp.gate_proj', 'model.layers.2.mlp.up_proj', 'model.layers.2.mlp.down_proj', 'model.layers.24.self_attn.q_proj', 'model.layers.24.self_attn.k_proj', 'model.layers.24.self_attn.v_proj', 'model.layers.24.self_attn.o_proj', 'model.layers.27.mlp.up_proj', 'lm_head']. Updating config.
Unsloth: Saving merged 4bit model to qwen3-0.6b-lora-merged...
Unsloth: Merged 4bit model saved.
Unsloth: Merged 4bit model process completed.


In [ ]:
from unsloth import FastLanguageModel
from peft import PeftModel
import torch

BASE_MODEL = "unsloth/Qwen3-0.6B-unsloth-bnb-4bit" # Replace with your actual base model path
LORA_PATH = "/content/ruozhi-Qwen3-0.6B" # Replace with the directory where your LoRA adapter is saved
MERGED_FP16_DIR = "/content/Qwen3-0.6B-merged-fp16" # Directory to save the merged 16-bit model
MERGED_4BIT_DIR = "/content/Qwen3-0.6B-merged-4bit" # Directory to save the merged 4-bit model (optional)


# Load the base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL,
    max_seq_length = 2048, # Adjust as needed
    dtype = None, # None + load_in_4bit=True => 4bit加载
    load_in_4bit = True,
    device_map = "auto",
    trust_remote_code = True,
)

# Load the LoRA adapter
model = PeftModel.from_pretrained(
    model,
    LORA_PATH,
    is_trainable=False,
)

# Merge LoRA adapter and save as 16-bit model (recommended)
import os
os.makedirs(MERGED_FP16_DIR, exist_ok=True)
model.save_pretrained_merged(
    MERGED_FP16_DIR,
    tokenizer = tokenizer,
    save_method = "merged_16bit", # Recommended to export as 16-bit
    max_shard_size = "2GB",
)
print(f"[OK] Merged FP16 model saved to: {MERGED_FP16_DIR}")

# Optional: Merge LoRA adapter and save as 4-bit model
os.makedirs(MERGED_4BIT_DIR, exist_ok=True)
model.save_pretrained_merged(
    MERGED_4BIT_DIR,
    tokenizer = tokenizer,
    save_method = "merged_4bit_forced", # Use forced if you are sure
    max_shard_size = "2GB",
)
print(f"[OK] Merged 4bit model saved to: {MERGED_4BIT_DIR}")

# Example of how to load and use the merged model
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_path = MERGED_FP16_DIR # Or MERGED_4BIT_DIR
# tok = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
# model = AutoModelForCausalLM.from_pretrained(
#     model_path,
#     torch_dtype = torch.bfloat16 if (torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8) else torch.float16,
#     device_map = "auto",
#     trust_remote_code = True,
# )

# if tok.pad_token is None:
#     tok.pad_token = tok.eos_token
# if getattr(model.config, "pad_token_id", None) is None:
#     model.config.pad_token_id = tok.pad_token_id

# prompt = "What is the capital of France?" # Replace with your prompt
# inputs = tok(prompt, return_tensors="pt").to(model.device)
# with torch.no_grad():
#     out = model.generate(
#         **inputs,
#         max_new_tokens=64,
#         do_sample=False,
#         eos_token_id=tok.eos_token_id,
#         pad_token_id=tok.pad_token_id,
#     )
# print(tok.decode(out[0], skip_special_tokens=True))

In [ ]:
from unsloth import FastLanguageModel
from peft import PeftModel
import torch

BASE_MODEL = "unsloth/Qwen3-0.6B-unsloth-bnb-4bit" # Replace with your actual base model path
LORA_PATH = "/content/ruozhi-Qwen3-0.6B" # Replace with the directory where your LoRA adapter is saved

# Load the base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_MODEL,
    max_seq_length = 2048, # Adjust as needed
    dtype = None, # None + load_in_4bit=True => 4bit加载
    load_in_4bit = True,
    device_map = "auto",
    trust_remote_code = True,
)

# Load the LoRA adapter
model = PeftModel.from_pretrained(
    model,
    LORA_PATH,
    is_trainable=False,
)

# Set model to inference mode
FastLanguageModel.for_inference(model)

# Define the ask_catgirl function using the base model with LoRA and tokenizer
def ask_catgirl(question):
  messages = [
    {"role" : "user", "content" : question}
]
  text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True,
    enable_thinking = False, # 思考模式
)

  from transformers import TextStreamer
  _ = model.generate(
      **tokenizer(text, return_tensors = "pt").to("cuda"),
      max_new_tokens = 256, # 输出长度
      temperature = 0.7, top_p = 0.8, top_k = 20,
      streamer = TextStreamer(tokenizer, skip_prompt = True),
  )

# Use the ask_catgirl function with an example input
ask_catgirl("TASK_GSDP_SORTINGCENTER_UNREACHABLE_RETURN_PROCESS")